# Question 6: Persistance et érosion de la ségrégation par cohorte

## Basé sur Sung et al. (2018)

### Contexte

Sung et al. (2018) ont établi que les étudiants de première année (Freshmen) tendent à former des communautés topologiquement **plus denses** que leurs aînés. Cependant, les auteurs suggèrent que cet effet s'estompe lorsque l'on considère des structures **non-chevauchantes** (non-overlapping).

### Question de recherche

**"Si l'on applique un algorithme de partitionnement strict, l'homophilie basée sur l'année persiste-t-elle sous la forme d'une forte homogénéité démographique ? Plus spécifiquement, observons-nous une diminution significative du Dominance Ratio de l'année au sein des communautés à mesure que les étudiants avancent dans leur cursus (Freshmen vs Seniors) ?"**

### Hypothèse

Nous formulons l'hypothèse d'une **intégration sociale progressive** :

1. **H1** : Les communautés dominées par les **Freshmen** afficheront un **Dominance Ratio très élevé** (communautés "pures"), indiquant un cercle social exclusif

2. **H2** : Les communautés dominées par les **Seniors** afficheront un **Dominance Ratio significativement plus faible**, reflétant une ouverture aux autres promotions

### Métrique : Dominance Ratio

Pour quantifier la "pureté" d'une communauté :

$$DominanceRatio(C) = \frac{\max_{y \in Years} |\{n \in C : \text{year}(n) = y\}|}{|C|}$$

Cette mesure représente la **proportion de la classe majoritaire** dans une communauté.

---

## Méthodologie

1. **Algorithme** : Louvain (partitionnement strict, non-chevauchant)
2. **Calcul** : Dominance Ratio pour chaque communauté
3. **Classification** : Identifier les communautés "Freshmen-dominant" vs "Senior-dominant"
4. **Comparaison** : Test statistique (t-test) pour valider H1 et H2

In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from tqdm import tqdm
from collections import Counter
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

## Question 6(a): Formulation détaillée

### Sujet

Persistance et érosion de la ségrégation par cohorte dans un partitionnement strict.

### Question de Recherche

Sung et al. (2018) ont établi que les étudiants de première année (Freshmen, classe 2010) tendent à former des communautés topologiquement plus denses que leurs aînés. Cependant, les auteurs suggèrent que cet effet s'estompe lorsque l'on considère des structures non-chevauchantes (non-overlapping).

**Notre étude vise à tester la robustesse de cette ségrégation sociale** : Si l'on applique un algorithme de partitionnement strict, l'homophilie basée sur l'année persiste-t-elle sous la forme d'une forte homogénéité démographique ? Plus spécifiquement, observons-nous une diminution significative du "Dominance Ratio" (ratio de dominance) de l'année au sein des communautés à mesure que les étudiants avancent dans leur cursus (Freshmen vs Seniors) ?

### Hypothèse

En nous basant sur le phénomène de cohorte identifié par Sung et al., nous formulons l'hypothèse d'une **intégration sociale progressive**.

Bien que le partitionnement strict (non-chevauchant) puisse masquer les variations de densité topologique, nous prédisons qu'il révélera une forte ségrégation sociale pour les nouveaux arrivants. Concrètement :

1. **H1 (Ségrégation Freshmen)** : Les communautés dominées par les Freshmen afficheront un **Dominance Ratio très élevé** (>0.80), indiquant un cercle social exclusif ("tribu" freshman).

2. **H2 (Intégration Seniors)** : Les communautés dominées par les Seniors afficheront un **Dominance Ratio significativement plus faible** (<0.65), reflétant une ouverture aux autres promotions et une intégration sociale accrue.

### Métrique : Dominance Ratio

Pour quantifier cela, nous réutilisons la métrique du papier, le **Dominance Ratio**, que nous appliquons ici comme mesure de pureté de la classe majoritaire au sein d'une partition Louvain :

$$DominanceRatio(C) = \frac{\max_{y \in Years} |\{n \in C : \text{year}(n) = y\}|}{|C|}$$

Cette mesure nous permettra de valider si l'effet "tribu" des Freshmen survit à la contrainte algorithmique d'appartenance unique.

### Justification théorique

**Théorie de la socialisation progressive** :
- Les Freshmen arrivent sur le campus sans réseau social préexistant → forte homophilie de cohorte
- Au fil des années, accumulation de liens cross-year via cours, activités, résidences
- Les Seniors ont eu 3-4 ans pour diversifier leurs cercles sociaux

**Sung et al. (2018)** ont montré cet effet avec des méthodes chevauchantes (overlapping communities). Notre contribution : **tester si cet effet persiste avec un partitionnement strict** (Louvain).

### Prédictions quantitatives

Basé sur Sung et al. et nos propres observations du dataset :

```
Communautés Freshmen-dominant:
  Dominance Ratio attendu: 0.85 ± 0.10
  Interprétation: 85% de la communauté est de la même année

Communautés Senior-dominant:
  Dominance Ratio attendu: 0.60 ± 0.15
  Interprétation: Seulement 60% de la même année, 40% mixte

Différence attendue: 0.25 (p < 0.01)
```

## Question 6(b): Méthodologie et code

### Configuration

In [7]:
# Configuration
data_dir = Path("data")

# Universités sélectionnées
universities = [
    'Caltech36.gml',
    'Haverford76.gml',
    'MIT8.gml',
    'Johns Hopkins55.gml'
]

# Définition des classes (selon le dataset FB100, les années sont encodées 2005-2010)
# Ces valeurs peuvent varier selon l'université - nous allons les détecter automatiquement
FRESHMAN_YEARS = [2009, 2010]  # Les années les plus récentes
SENIOR_YEARS = [2005, 2006, 2007]  # Les années les plus anciennes

# Taille minimale de communauté pour l'analyse
MIN_COMMUNITY_SIZE = 10

print(f"Configuration:")
print(f"  Universités: {len(universities)}")
print(f"  Freshman years: {FRESHMAN_YEARS}")
print(f"  Senior years: {SENIOR_YEARS}")
print(f"  Taille min communauté: {MIN_COMMUNITY_SIZE}")

Configuration:
  Universités: 4
  Freshman years: [2009, 2010]
  Senior years: [2005, 2006, 2007]
  Taille min communauté: 10


### Algorithme de détection : Louvain

In [3]:
# Importer Louvain
try:
    from sknetwork.clustering import Louvain
    LOUVAIN_AVAILABLE = True
    print("✓ scikit-network disponible")
except ImportError:
    print("⚠️ scikit-network non disponible, utilisation de NetworkX Louvain")
    LOUVAIN_AVAILABLE = False

⚠️ scikit-network non disponible, utilisation de NetworkX Louvain


In [4]:
def detect_communities_louvain(G):
    """
    Détection de communautés avec Louvain
    Partitionnement STRICT (non-chevauchant)
    
    Returns:
        dict: {node_id: community_label}
    """
    if LOUVAIN_AVAILABLE:
        from sknetwork.data import from_networkx
        adjacency = from_networkx(G)
        louvain = Louvain()
        labels_array = louvain.fit_transform(adjacency.adjacency)
        node_list = list(G.nodes())
        labels_dict = {node_list[i]: int(labels_array[i]) for i in range(len(node_list))}
    else:
        import networkx.algorithms.community as nx_comm
        communities = nx_comm.louvain_communities(G, seed=42)
        labels_dict = {}
        for idx, community in enumerate(communities):
            for node in community:
                labels_dict[node] = idx
    
    return labels_dict

### Fonctions de calcul du Dominance Ratio

In [5]:
def calculate_dominance_ratio(G, community_nodes):
    """
    Calcule le Dominance Ratio d'une communauté sur l'attribut 'year'
    
    DominanceRatio(C) = max_y |{n ∈ C : year(n) = y}| / |C|
    
    Returns:
        tuple: (dominance_ratio, dominant_year, year_counts)
    """
    # Extraire les années (en excluant 0 = missing)
    years = [G.nodes[node].get('year', 0) for node in community_nodes]
    years = [y for y in years if y != 0]
    
    if len(years) < 3:  # Pas assez de données
        return np.nan, None, {}
    
    # Compter les occurrences
    year_counts = Counter(years)
    
    # Année dominante et son compte
    dominant_year, max_count = year_counts.most_common(1)[0]
    
    # Dominance Ratio
    dominance_ratio = max_count / len(years)
    
    return dominance_ratio, dominant_year, dict(year_counts)

def classify_community_by_dominant_year(dominant_year, freshman_years, senior_years):
    """
    Classifie une communauté selon l'année dominante
    
    Returns:
        str: 'Freshman', 'Senior', 'Sophomore', 'Junior', ou 'Mixed'
    """
    if dominant_year is None:
        return 'Unknown'
    
    if dominant_year in freshman_years:
        return 'Freshman'
    elif dominant_year in senior_years:
        return 'Senior'
    else:
        # Années intermédiaires (Sophomore/Junior)
        return 'Intermediate'

def get_year_statistics(G):
    """
    Obtenir les statistiques sur les années dans le réseau
    pour détecter automatiquement Freshman/Senior years
    
    Returns:
        dict: statistiques sur les années
    """
    years = [G.nodes[node].get('year', 0) for node in G.nodes()]
    years = [y for y in years if y != 0]
    
    if not years:
        return {}
    
    year_counts = Counter(years)
    
    return {
        'min_year': min(years),
        'max_year': max(years),
        'year_counts': dict(year_counts),
        'n_years': len(year_counts)
    }

### Analyse complète

In [11]:
# Structure pour stocker tous les résultats
all_community_results = []
university_graphs = {}
university_communities = {}

# Seed
np.random.seed(42)

print("="*80)
print("ANALYSE - DOMINANCE RATIO PAR COHORTE")
print("="*80)

for univ_file in universities:
    univ_name = univ_file.replace('.gml', '')
    
    print(f"\n{'='*80}")
    print(f"UNIVERSITÉ: {univ_name}")
    print(f"{'='*80}")
    
    # Charger
    G = nx.read_gml(data_dir / univ_file)
    print(f"Nœuds: {G.number_of_nodes()}, Arêtes: {G.number_of_edges()}")
    
    # LCC
    if not nx.is_connected(G):
        largest_cc = max(nx.connected_components(G), key=len)
        G = G.subgraph(largest_cc).copy()
        print(f"LCC: {G.number_of_nodes()} nœuds, {G.number_of_edges()} arêtes")
    
    university_graphs[univ_name] = G
    
    # Statistiques sur les années
    year_stats = get_year_statistics(G)
    print(f"\nStatistiques années:")
    print(f"  Min year: {year_stats.get('min_year', 'N/A')}")
    print(f"  Max year: {year_stats.get('max_year', 'N/A')}")
    print(f"  Nombre d'années distinctes: {year_stats.get('n_years', 'N/A')}")
    if 'year_counts' in year_stats:
        print(f"  Distribution: {year_stats['year_counts']}")
    
    # Ajuster freshman/senior years selon cette université
    if 'max_year' in year_stats:
        max_year = year_stats['max_year']
        min_year = year_stats['min_year']
        
        # Freshmen = 2 années les plus récentes
        freshman_years_univ = [max_year, max_year - 1]
        print(freshman_years_univ)
        # Seniors = 3 années les plus anciennes
        senior_years_univ = [min_year, min_year + 1, min_year + 2]
        
        print(f"\nAjustement pour cette université:")
        print(f"  Freshman years: {freshman_years_univ}")
        print(f"  Senior years: {senior_years_univ}")
    else:
        freshman_years_univ = FRESHMAN_YEARS
        senior_years_univ = SENIOR_YEARS
    
    # Détection avec Louvain
    print(f"\n--- Détection de communautés: Louvain (partitionnement strict) ---")
    communities = detect_communities_louvain(G)
    n_communities = len(set(communities.values()))
    print(f"Nombre de communautés: {n_communities}")
    
    # Grouper par communauté
    comm_to_nodes = {}
    for node, comm_id in communities.items():
        if comm_id not in comm_to_nodes:
            comm_to_nodes[comm_id] = []
        comm_to_nodes[comm_id].append(node)
    
    # Analyser chaque communauté
    print(f"\n--- Calcul du Dominance Ratio ---")
    
    freshman_communities = []
    senior_communities = []
    intermediate_communities = []
    
    for comm_id, nodes in comm_to_nodes.items():
        size = len(nodes)
        
        if size < MIN_COMMUNITY_SIZE:
            continue
        
        # Calculer Dominance Ratio
        dominance_ratio, dominant_year, year_counts = calculate_dominance_ratio(G, nodes)
        
        if np.isnan(dominance_ratio):
            continue
        
        # Classifier
        comm_type = classify_community_by_dominant_year(dominant_year, freshman_years_univ, senior_years_univ)
        
        # Stocker
        result = {
            'university': univ_name,
            'community_id': comm_id,
            'size': size,
            'dominant_year': dominant_year,
            'dominance_ratio': dominance_ratio,
            'type': comm_type,
            'year_counts': year_counts
        }
        all_community_results.append(result)
        
        # Grouper par type
        if comm_type == 'Freshman':
            freshman_communities.append(result)
        elif comm_type == 'Senior':
            senior_communities.append(result)
        else:
            intermediate_communities.append(result)
    
    print(f"\nRésumé:")
    print(f"  Communautés Freshman-dominant: {len(freshman_communities)}")
    print(f"  Communautés Senior-dominant: {len(senior_communities)}")
    print(f"  Communautés Intermediate: {len(intermediate_communities)}")
    
    if freshman_communities:
        avg_dr_fresh = np.mean([c['dominance_ratio'] for c in freshman_communities])
        print(f"  Dominance Ratio moyen (Freshman): {avg_dr_fresh:.3f}")
    
    if senior_communities:
        avg_dr_senior = np.mean([c['dominance_ratio'] for c in senior_communities])
        print(f"  Dominance Ratio moyen (Senior): {avg_dr_senior:.3f}")
    
    university_communities[univ_name] = communities

print("\n" + "="*80)
print("✓ ANALYSE TERMINÉE")
print("="*80)

ANALYSE - DOMINANCE RATIO PAR COHORTE

UNIVERSITÉ: Caltech36
Nœuds: 769, Arêtes: 16656
LCC: 762 nœuds, 16651 arêtes

Statistiques années:
  Min year: 1968
  Max year: 2010
  Nombre d'années distinctes: 15
  Distribution: {2008: 173, 2006: 152, 2005: 105, 2007: 133, 2009: 23, 2004: 39, 2003: 10, 1968: 1, 2002: 7, 2001: 2, 2010: 2, 1996: 1, 1984: 1, 1999: 1, 1993: 1}
[2010, 2009]

Ajustement pour cette université:
  Freshman years: [2010, 2009]
  Senior years: [1968, 1969, 1970]

--- Détection de communautés: Louvain (partitionnement strict) ---
Nombre de communautés: 9

--- Calcul du Dominance Ratio ---

Résumé:
  Communautés Freshman-dominant: 1
  Communautés Senior-dominant: 0
  Communautés Intermediate: 8
  Dominance Ratio moyen (Freshman): 0.812

UNIVERSITÉ: Haverford76
Nœuds: 1446, Arêtes: 59589

Statistiques années:
  Min year: 1938
  Max year: 2009
  Nombre d'années distinctes: 13
  Distribution: {2005: 176, 2008: 297, 2007: 229, 2009: 288, 2004: 81, 2006: 225, 2003: 28, 2000: 4,

### Résultats consolidés

In [9]:
# DataFrame
df_results = pd.DataFrame(all_community_results)

# Sauvegarder
output_dir = Path('../outputs/question6')
output_dir.mkdir(parents=True, exist_ok=True)
df_results.to_csv(output_dir / 'dominance_ratio_results.csv', index=False)
print("✓ Résultats sauvegardés\n")

# Afficher
print("="*80)
print("RÉSULTATS CONSOLIDÉS")
print("="*80)
print(df_results[['university', 'community_id', 'size', 'dominant_year', 
                  'dominance_ratio', 'type']].to_string(index=False))

✓ Résultats sauvegardés

RÉSULTATS CONSOLIDÉS
     university  community_id  size  dominant_year  dominance_ratio         type
      Caltech36             0    90           2006         0.338710 Intermediate
      Caltech36             1    88           2006         0.236111 Intermediate
      Caltech36             2    82           2008         0.324324 Intermediate
      Caltech36             3    17           2009         0.812500     Freshman
      Caltech36             4    27           2008         0.833333 Intermediate
      Caltech36             5    96           2008         0.279070 Intermediate
      Caltech36             6   122           2007         0.320388 Intermediate
      Caltech36             7   115           2006         0.313725 Intermediate
      Caltech36             8   125           2008         0.303571 Intermediate
    Haverford76             0   293           2009         0.993056     Freshman
    Haverford76             1   314           2008         0.86

### Tests statistiques - VALIDATION DES HYPOTHÈSES

In [10]:
print("\n" + "="*80)
print("TESTS STATISTIQUES - H1 ET H2")
print("="*80)

# Filtrer les communautés Freshman et Senior
df_freshman = df_results[df_results['type'] == 'Freshman']
df_senior = df_results[df_results['type'] == 'Senior']

print(f"\nÉchantillon:")
print(f"  Communautés Freshman: {len(df_freshman)}")
print(f"  Communautés Senior: {len(df_senior)}")

if len(df_freshman) > 0 and len(df_senior) > 0:
    
    # Extraire les Dominance Ratios
    dr_freshman = df_freshman['dominance_ratio'].values
    dr_senior = df_senior['dominance_ratio'].values
    
    # ========================================================================
    # H1: Les communautés Freshman ont un Dominance Ratio ÉLEVÉ
    # ========================================================================
    print(f"\n{'='*80}")
    print("H1: Ségrégation Freshman (Dominance Ratio élevé)")
    print(f"{'='*80}")
    
    mean_dr_freshman = dr_freshman.mean()
    std_dr_freshman = dr_freshman.std()
    
    print(f"Dominance Ratio (Freshman): {mean_dr_freshman:.3f} (±{std_dr_freshman:.3f})")
    print(f"Min: {dr_freshman.min():.3f}, Max: {dr_freshman.max():.3f}")
    
    # Test one-sample: Est-ce que DR_freshman > 0.70 ?
    threshold_high = 0.70
    t_stat_fresh, p_value_fresh = stats.ttest_1samp(dr_freshman, threshold_high, alternative='greater')
    
    print(f"\nTest: DR_freshman > {threshold_high} ?")
    print(f"t-statistic: {t_stat_fresh:.4f}")
    print(f"p-value: {p_value_fresh:.4f}")
    
    if p_value_fresh < 0.05:
        print(f"\n✓✓✓ H1 CONFIRMÉE (p < 0.05) ✓✓✓")
        print(f"Les communautés Freshman ont un Dominance Ratio significativement > {threshold_high}")
        print(f"→ FORTE SÉGRÉGATION par cohorte pour les nouveaux arrivants")
    else:
        print(f"\n≈ H1 NON CONFIRMÉE (p ≥ 0.05)")
    
    # ========================================================================
    # H2: Les communautés Senior ont un Dominance Ratio PLUS FAIBLE
    # ========================================================================
    print(f"\n{'='*80}")
    print("H2: Intégration Senior (Dominance Ratio plus faible)")
    print(f"{'='*80}")
    
    mean_dr_senior = dr_senior.mean()
    std_dr_senior = dr_senior.std()
    
    print(f"Dominance Ratio (Senior): {mean_dr_senior:.3f} (±{std_dr_senior:.3f})")
    print(f"Min: {dr_senior.min():.3f}, Max: {dr_senior.max():.3f}")
    
    # Test de comparaison: DR_freshman > DR_senior ?
    print(f"\nComparaison Freshman vs Senior:")
    print(f"  Différence: {mean_dr_freshman - mean_dr_senior:.3f}")
    
    # T-test indépendant (one-tailed)
    if len(dr_freshman) > 1 and len(dr_senior) > 1:
        t_stat, p_value = stats.ttest_ind(dr_freshman, dr_senior, alternative='greater')
        
        print(f"\nTest: DR_freshman > DR_senior ?")
        print(f"t-statistic: {t_stat:.4f}")
        print(f"p-value: {p_value:.4f}")
        
        if p_value < 0.05:
            print(f"\n✓✓✓ H2 CONFIRMÉE (p < 0.05) ✓✓✓")
            print(f"Les Seniors ont un Dominance Ratio significativement PLUS FAIBLE que les Freshmen")
            print(f"→ INTÉGRATION SOCIALE PROGRESSIVE confirmée")
            print(f"→ Effet 'tribu' freshman s'estompe avec l'avancement dans le cursus")
        else:
            print(f"\n≈ H2 NON CONFIRMÉE (p ≥ 0.05)")
    
    # ========================================================================
    # Effect size (Cohen's d)
    # ========================================================================
    print(f"\n{'='*80}")
    print("EFFECT SIZE (Cohen's d)")
    print(f"{'='*80}")
    
    pooled_std = np.sqrt((std_dr_freshman**2 + std_dr_senior**2) / 2)
    cohens_d = (mean_dr_freshman - mean_dr_senior) / pooled_std
    
    print(f"Cohen's d: {cohens_d:.3f}")
    
    if abs(cohens_d) < 0.2:
        effect_interpretation = "petit effet"
    elif abs(cohens_d) < 0.5:
        effect_interpretation = "effet moyen"
    elif abs(cohens_d) < 0.8:
        effect_interpretation = "effet important"
    else:
        effect_interpretation = "effet très important"
    
    print(f"Interprétation: {effect_interpretation}")
    
else:
    print("\n⚠️ Échantillon insuffisant pour les tests statistiques")

print("\n" + "="*80)


TESTS STATISTIQUES - H1 ET H2

Échantillon:
  Communautés Freshman: 6
  Communautés Senior: 0

⚠️ Échantillon insuffisant pour les tests statistiques



### Visualisation 1: Distribution des Dominance Ratios

In [ ]:
# Filtrer pour visualisation
df_plot = df_results[df_results['type'].isin(['Freshman', 'Senior', 'Intermediate'])]

fig, ax = plt.subplots(figsize=(12, 6))

# Violin plot
positions = {'Freshman': 1, 'Intermediate': 2, 'Senior': 3}
colors = {'Freshman': '#e74c3c', 'Intermediate': '#f39c12', 'Senior': '#3498db'}

for type_name in ['Freshman', 'Intermediate', 'Senior']:
    data = df_plot[df_plot['type'] == type_name]['dominance_ratio'].values
    if len(data) > 0:
        parts = ax.violinplot([data], positions=[positions[type_name]], widths=0.7,
                              showmeans=True, showmedians=True)
        for pc in parts['bodies']:
            pc.set_facecolor(colors[type_name])
            pc.set_alpha(0.7)

# Ligne de référence
ax.axhline(y=0.70, color='gray', linestyle='--', linewidth=1.5, alpha=0.7, 
          label='Seuil théorique (0.70)')

# Annotations des moyennes
for type_name in ['Freshman', 'Intermediate', 'Senior']:
    data = df_plot[df_plot['type'] == type_name]['dominance_ratio'].values
    if len(data) > 0:
        mean_val = data.mean()
        ax.text(positions[type_name], mean_val + 0.05, f'{mean_val:.2f}',
               ha='center', fontsize=11, fontweight='bold')

ax.set_ylabel('Dominance Ratio', fontsize=12)
ax.set_title('Distribution du Dominance Ratio par type de cohorte\n(Partitionnement strict - Louvain)', 
            fontsize=14)
ax.set_xticks([1, 2, 3])
ax.set_xticklabels(['Freshman', 'Intermediate\n(Soph./Junior)', 'Senior'], fontsize=11)
ax.set_ylim(0, 1)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(output_dir / 'dominance_ratio_distribution.png', dpi=300, bbox_inches='tight')
plt.show()
print("✓ Graphique 1 sauvegardé")

### Visualisation 2: Comparaison par université

In [ ]:
# Moyennes par université et type
summary = df_results.groupby(['university', 'type'])['dominance_ratio'].agg(['mean', 'count']).reset_index()
summary = summary[summary['type'].isin(['Freshman', 'Senior'])]

fig, ax = plt.subplots(figsize=(12, 6))

# Pour chaque université
universities_plot = summary['university'].unique()
x = np.arange(len(universities_plot))
width = 0.35

freshman_means = []
senior_means = []

for univ in universities_plot:
    univ_data = summary[summary['university'] == univ]
    
    fresh_data = univ_data[univ_data['type'] == 'Freshman']
    senior_data = univ_data[univ_data['type'] == 'Senior']
    
    freshman_means.append(fresh_data['mean'].values[0] if len(fresh_data) > 0 else 0)
    senior_means.append(senior_data['mean'].values[0] if len(senior_data) > 0 else 0)

ax.bar(x - width/2, freshman_means, width, label='Freshman', alpha=0.8, color='#e74c3c')
ax.bar(x + width/2, senior_means, width, label='Senior', alpha=0.8, color='#3498db')

# Ligne de référence
ax.axhline(y=0.70, color='gray', linestyle='--', linewidth=1.5, alpha=0.5)

ax.set_ylabel('Dominance Ratio moyen', fontsize=12)
ax.set_title('Dominance Ratio par université et cohorte', fontsize=14)
ax.set_xticks(x)
ax.set_xticklabels(universities_plot, rotation=45, ha='right', fontsize=10)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3, axis='y')
ax.set_ylim(0, 1)

plt.tight_layout()
plt.savefig(output_dir / 'dominance_ratio_by_university.png', dpi=300, bbox_inches='tight')
plt.show()
print("✓ Graphique 2 sauvegardé")

### Visualisation 3: Scatter plot (Taille vs Dominance Ratio)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

# Séparer par type
for type_name, color in [('Freshman', '#e74c3c'), ('Senior', '#3498db'), ('Intermediate', '#f39c12')]:
    data = df_results[df_results['type'] == type_name]
    if len(data) > 0:
        ax.scatter(data['size'], data['dominance_ratio'], 
                  s=100, alpha=0.6, c=color, label=type_name, edgecolors='black')

# Ligne de référence
ax.axhline(y=0.70, color='gray', linestyle='--', linewidth=1.5, alpha=0.5, 
          label='Seuil 0.70')

ax.set_xlabel('Taille de la communauté', fontsize=12)
ax.set_ylabel('Dominance Ratio', fontsize=12)
ax.set_title('Relation entre taille de communauté et Dominance Ratio', fontsize=13)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
ax.set_ylim(0, 1)

plt.tight_layout()
plt.savefig(output_dir / 'scatter_size_vs_dominance.png', dpi=300, bbox_inches='tight')
plt.show()
print("✓ Graphique 3 sauvegardé")

### Visualisation 4: Heatmap (Université × Type)

In [ ]:
# Pivot table
pivot = df_results.pivot_table(values='dominance_ratio', 
                               index='university', 
                               columns='type', 
                               aggfunc='mean')

# Réordonner les colonnes
column_order = [col for col in ['Freshman', 'Intermediate', 'Senior'] if col in pivot.columns]
pivot = pivot[column_order]

fig, ax = plt.subplots(figsize=(10, 6))
sns.heatmap(pivot, annot=True, fmt='.2f', cmap='RdYlBu_r', 
            vmin=0, vmax=1, cbar_kws={'label': 'Dominance Ratio'}, ax=ax,
            linewidths=0.5, linecolor='gray')

ax.set_title('Dominance Ratio moyen par université et cohorte', fontsize=13)
ax.set_xlabel('Type de cohorte', fontsize=11)
ax.set_ylabel('Université', fontsize=11)

plt.tight_layout()
plt.savefig(output_dir / 'heatmap_dominance_ratio.png', dpi=300, bbox_inches='tight')
plt.show()
print("✓ Graphique 4 sauvegardé")

## Question 6(c): Interprétation et conclusion

### Synthèse des résultats

**[À COMPLÉTER APRÈS EXÉCUTION]**

#### Validation de H1 (Ségrégation Freshman)

**Résultat** : [insérer résultat du test]

**Interprétation si H1 confirmée (p < 0.05)** :
> "Nos résultats confirment que les communautés dominées par les Freshmen présentent un Dominance Ratio significativement élevé (moyenne = X.XX, p < 0.05). Cela valide l'existence d'un effet 'tribu' freshman même sous contrainte de partitionnement strict (Louvain). Contrairement aux préoccupations de Sung et al. (2018) sur l'effet des méthodes non-chevauchantes, la ségrégation par cohorte **persiste** et reste détectable algorithmiquement."

**Interprétation si H1 infirmée** :
> "Le Dominance Ratio des communautés Freshman n'est pas significativement supérieur au seuil de 0.70. Cela suggère que le partitionnement strict (Louvain) 'dilue' l'effet de ségrégation observé par Sung et al. avec des méthodes chevauchantes. L'effet 'tribu' freshman existe peut-être en densité locale, mais ne se traduit pas en pureté démographique absolue."

---

#### Validation de H2 (Intégration Senior)

**Résultat** : [insérer résultat du test]

**Interprétation si H2 confirmée (p < 0.05)** :
> "Nous observons une **diminution significative** du Dominance Ratio entre les communautés Freshman et Senior (différence = X.XX, p < 0.05, Cohen's d = X.XX). Ce résultat valide l'hypothèse d'**intégration sociale progressive** : les étudiants diversifient leurs cercles sociaux au fil de leur cursus. L'effet 'tribu' s'érode naturellement avec le temps, même dans un partitionnement algorithmique strict."

**Interprétation si H2 infirmée** :
> "Aucune différence significative n'est observée entre Freshmen et Seniors. Cela suggère soit (1) que la ségrégation par cohorte persiste tout au long du cursus, soit (2) que le Dominance Ratio ne capture pas les nuances d'intégration sociale (ex: les Seniors peuvent avoir des liens cross-year faibles mais nombreux)."

---

### Discussion approfondie

#### 1. Robustesse de la ségrégation vs méthode de détection

**Sung et al. (2018)** ont utilisé des méthodes de détection de communautés **chevauchantes** (overlapping) qui permettent à un étudiant d'appartenir à plusieurs groupes. Ils ont observé que les Freshmen forment des cliques denses.

**Notre contribution** : En imposant un **partitionnement strict** (Louvain), nous testons si cet effet survit à une contrainte plus forte. Si H1 est confirmée, cela signifie que la ségrégation freshman est **robuste** et ne dépend pas uniquement de la possibilité d'appartenir à plusieurs groupes.

**Implications** :
- **Si confirmée** : La ségrégation est structurelle, pas un artefact méthodologique
- **Si infirmée** : La ségrégation est plus subtile, visible en chevauchement mais pas en partition stricte

#### 2. Mécanismes de l'intégration sociale

Si H2 est confirmée (DR_senior < DR_freshman), plusieurs mécanismes peuvent l'expliquer :

1. **Accumulation temporelle** : 3-4 ans pour créer des liens cross-year via cours avancés, stages, activités

2. **Contraintes structurelles évolutives** :
   - Freshmen : assignés au même dorm → contacts limités à leur cohorte
   - Seniors : logements off-campus, cours de spécialisation → contacts diversifiés

3. **Maturité sociale** : Les Seniors ont développé une identité sociale moins dépendante de leur cohorte

4. **Survival bias** : Les étudiants qui restent jusqu'en Senior year sont peut-être ceux qui s'intègrent mieux (les autres ont peut-être quitté)

#### 3. Variabilité institutionnelle

[OBSERVER les différences entre universités]

Exemple d'interprétation :
> "Caltech montre un DR_freshman = 0.XX vs MIT = 0.YY. Cette différence peut s'expliquer par [taille du campus / système de Houses / politiques de logement]. Les petites institutions (Caltech, Haverford) favorisent peut-être plus la cohésion de cohorte."

#### 4. Limitations

1. **Snapshot temporel** : Données à un instant T. Nous ne pouvons pas suivre l'évolution individuelle des étudiants

2. **Causalité** : Corrélation observée, pas causalité. Est-ce le temps qui cause l'intégration ou d'autres facteurs ?

3. **Définition de Freshman/Senior** : Basée sur l'année déclarée, peut ne pas correspondre exactement au statut réel (transferts, redoublements)

4. **Autres facteurs** : Le Dominance Ratio ne capture qu'une dimension (year). D'autres attributs (dorm, major) peuvent aussi structurer

---

### Conclusion générale

**Notre question** : La ségrégation par cohorte identifiée par Sung et al. persiste-t-elle sous partitionnement strict ?

**Verdict** : [À COMPLÉTER]

**Scénario A (H1 et H2 confirmées - SUCCÈS COMPLET)** :
> "Nos résultats confirment et étendent les observations de Sung et al. (2018). Nous démontrons que : (1) La ségrégation freshman est **robuste** et détectable même avec un partitionnement strict (DR_freshman significativement > 0.70), et (2) Cette ségrégation **s'érode progressivement** avec l'avancement dans le cursus (DR_senior < DR_freshman, p < 0.05). Cette double validation établit que l'effet 'tribu' freshman n'est pas un artefact des méthodes de détection chevauchantes, mais reflète une réalité sociologique fondamentale des réseaux universitaires."

**Scénario B (H1 confirmée, H2 infirmée)** :
> "La ségrégation freshman est confirmée (DR élevé), mais l'intégration Senior ne l'est pas. Cela suggère que si les nouveaux arrivants forment bien des 'tribus', cette tendance **ne s'estompe pas** avec le temps de la manière attendue. Hypothèses : (1) La ségrégation par cohorte persiste tout au long du cursus, ou (2) L'intégration prend des formes plus subtiles non capturées par le Dominance Ratio."

**Scénario C (H1 infirmée, H2 confirmée ou les deux infirmées)** :
> "Nos résultats suggèrent que l'effet de ségrégation observé par Sung et al. est **sensible à la méthode de détection**. Le partitionnement strict (Louvain) ne révèle pas de différences significatives de Dominance Ratio entre cohortes. Cela ne réfute pas l'existence de patterns sociaux par année, mais indique qu'ils sont plus complexes qu'une simple 'pureté démographique' de communautés."

---

### Perspectives futures

1. **Comparaison méthodologique** : Appliquer des méthodes chevauchantes (OSLOM, DEMON) et comparer les Dominance Ratios

2. **Analyse longitudinale** : Si données temporelles disponibles, suivre l'évolution du DR pour une même cohorte

3. **Facteurs prédictifs** : Quels attributs prédisent un DR élevé/faible ? (taille du dorm, popularité du major)

4. **Autres métriques** : Tester d'autres mesures de ségrégation (Entropy, Gini coefficient)

5. **Généralisation** : Tester sur d'autres universités, autres contextes (entreprises, réseaux en ligne)

---

### Contribution scientifique

Cette étude contribue à la littérature en :

1. **Testant la robustesse** des observations de Sung et al. sous une contrainte algorithmique différente

2. **Quantifiant** l'érosion de la ségrégation par cohorte avec une métrique claire (Dominance Ratio)

3. **Validant** (ou infirmant) l'hypothèse d'intégration sociale progressive dans un cadre de partitionnement strict

4. **Fournissant** des insights pour les politiques universitaires d'intégration sociale

---

## Résumé exécutif

### Question
> "La ségrégation par cohorte (effet 'tribu' freshman) persiste-t-elle sous partitionnement strict ?"

### Référence
Sung et al. (2018) - méthodes chevauchantes, densité topologique

### Notre approche
- **Algorithme** : Louvain (partitionnement strict)
- **Métrique** : Dominance Ratio (pureté démographique)
- **Hypothèses** : DR_freshman > 0.70 > DR_senior

### Résultats
[À compléter]

### Contribution
Test de robustesse de la ségrégation par cohorte + Quantification de l'intégration sociale progressive